# Loading Data

In [1]:
from datasets import load_dataset, load_from_disk

# tydiqa_gold = load_dataset("khalidalt/tydiqa-goldp", 'indonesian', trust_remote_code=True)
# mr_tydi = load_from_disk("./generated_data/mr_tydi_filled")

In [2]:
tydiqa_gold

DatasetDict({
    train: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 5702
    })
    validation: Dataset({
        features: ['id', 'language', 'document_title', 'passage_text', 'question_text', 'answers'],
        num_rows: 565
    })
})

# Data Cleaning

In [3]:
tydiqa_gold = tydiqa_gold.remove_columns(["language", "document_title", "passage_text"])
tydiqa_gold = tydiqa_gold.rename_column("id", "tydiqa_id")
tydiqa_gold = tydiqa_gold.rename_column("question_text", "query")

In [4]:
from datasets import DatasetDict

def clean_answers(example):
    example["answers"] = example["answers"]["text"]  # Ambil hanya bagian text
    return example

# Terapkan fungsi untuk membersihkan answers di setiap split
tydiqa_gold = DatasetDict({
    split: dataset.map(clean_answers)
    for split, dataset in tydiqa_gold.items()
})

In [5]:
import re
from datasets import load_from_disk, DatasetDict

# Fungsi untuk membersihkan teks: hapus newline & whitespace berlebih
def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

# Fungsi untuk membersihkan dan memilih jawaban terpendek
def clean_tydiqa(example):
    # Bersihkan query
    example["query"] = clean_text(example["query"])
    
    # Bersihkan answers dan pilih jawaban terpendek jika ada lebih dari satu
    cleaned_answers = [clean_text(ans) for ans in example["answers"]]
    example["answers"] = min(cleaned_answers, key=len) if cleaned_answers else ""  # Pilih jawaban terpendek

    return example

# Fungsi untuk membersihkan query di MR-TyDi
def clean_mr_tydi(example):
    example["query"] = clean_text(example["query"])
    return example

# Terapkan pembersihan pada dataset
tydiqa_gold_cleaned = DatasetDict({
    split: dataset.map(clean_tydiqa)
    for split, dataset in tydiqa_gold.items()
})

mr_tydi_cleaned = DatasetDict({
    split: dataset.map(clean_mr_tydi)
    for split, dataset in mr_tydi.items()
})

Map:   0%|          | 0/5702 [00:00<?, ? examples/s]

Map:   0%|          | 0/565 [00:00<?, ? examples/s]

In [6]:
check_if_answer_is_cleaned = tydiqa_gold_cleaned['train'].filter(lambda x: x['tydiqa_id']=="-2253919563477221294-3")
print(check_if_answer_is_cleaned[0]['answers'])

check_if_answer_more_than_1 = tydiqa_gold_cleaned['validation'].filter(lambda x: x['tydiqa_id']=="8601389648636013237-1")
check_if_answer_more_than_1[0]

Filter:   0%|          | 0/5702 [00:00<?, ? examples/s]

Nicolas Loufrani


Filter:   0%|          | 0/565 [00:00<?, ? examples/s]

{'tydiqa_id': '8601389648636013237-1',
 'query': 'siapakah ketua Perum LKBN pertama?',
 'answers': 'Mr. Soemanang'}

In [46]:
import re
from collections import Counter
from datasets import concatenate_datasets

# Gabungkan split train dan validation
tydiqa_gold_combined = concatenate_datasets([tydiqa_gold_cleaned["train"], tydiqa_gold_cleaned["validation"]])

# Fungsi untuk menghitung jumlah kata dalam jawaban
def count_words(text):
    return len(re.findall(r'\w+', text))

# Filter hanya row dengan answers lebih dari 10 kata
long_answer_rows = [row for row in tydiqa_gold_combined if count_words(row["answers"]) > 10]

# Ambil kata pertama dari query
first_words = [row["query"].split()[0] for row in long_answer_rows]

# Hitung jumlah kemunculan kata pertama dalam query
first_word_counts = Counter(first_words)

# Konversi ke daftar kata unik
unique_first_words = sorted(set(first_words))

# Cetak hasil
print(f"🔍 Ditemukan {len(unique_first_words)} kata unik yang menjadi kata pertama dalam query ketika answers memiliki lebih dari 10 kata.")
print("📋 Kata unik tersebut:", unique_first_words)


🔍 Ditemukan 42 kata unik yang menjadi kata pertama dalam query ketika answers memiliki lebih dari 10 kata.
📋 Kata unik tersebut: ['Aapakah', 'Ada', 'Apa', 'Apaka', 'Apakah', 'Bagaimana', 'Bagaimanakah', 'Berapa', 'Berapakah', 'Bidang', 'Dari', 'Darimanakah', 'Daun', 'Di', 'Dimana', 'Dimanakah', 'Dimankah', 'Kapan', 'Kenapa', 'Menceritakan', 'Mengapa', 'Musik', 'Siapa', 'Siapakah', 'Tentang', 'Untuk', 'Zat', 'aoakah', 'apa', 'apaah', 'apakah', 'berapa', 'berapakah', 'berdasarkan', 'dari', 'di', 'dimanakah', 'kapankah', 'mengapa', 'negara', 'siapakah', 'siapakh']


# Filtering untuk answers yang jumlah katanya kurang dari 15 (opsional)

# Join

In [57]:
tydiqa_gold_cleaned

DatasetDict({
    train: Dataset({
        features: ['tydiqa_id', 'query', 'answers'],
        num_rows: 5702
    })
    validation: Dataset({
        features: ['tydiqa_id', 'query', 'answers'],
        num_rows: 565
    })
})

In [7]:
from datasets import DatasetDict, concatenate_datasets

# Gabungkan split train & validation pada tydiqa_gold_cleaned
tydiqa_gold_combined = concatenate_datasets([tydiqa_gold_cleaned["train"], tydiqa_gold_cleaned["validation"]])

# Buat struktur baru mengikuti split dari mr_tydi_cleaned
joined_datasets = {}

for split, mr_tydi_split in mr_tydi_cleaned.items():
    # Buat dictionary {query: row} dari tydiqa_gold_cleaned untuk lookup cepat
    tydiqa_gold_dict = {row["query"]: row for row in tydiqa_gold_combined}
    
    # Buat daftar baru dengan menggabungkan informasi dari mr_tydi_cleaned dan tydiqa_gold_cleaned
    new_split_data = []
    
    for row in mr_tydi_split:
        query = row["query"]
        tydiqa_data = tydiqa_gold_dict.get(query, None)  # Ambil data dari tydiqa_gold jika ada
        
        # Gabungkan data (jika tidak ada di tydiqa_gold, biarkan bagian tersebut kosong)
        merged_row = {
            **row,  # Data dari mr_tydi_cleaned
            "tydiqa_id": tydiqa_data["tydiqa_id"] if tydiqa_data else None,
            "answers": tydiqa_data["answers"] if tydiqa_data else None
        }
        
        new_split_data.append(merged_row)

    # Konversi kembali ke Dataset
    joined_datasets[split] = mr_tydi_split.from_list(new_split_data)

# Simpan hasil sebagai DatasetDict
final_dataset = DatasetDict(joined_datasets)

# # Simpan hasil ke disk
# final_dataset.save_to_disk("generated_data/mr_tydi_tydiqa_joined")

print("✅ Dataset berhasil digabungkan berdasarkan `query` dengan struktur mengikuti `mr_tydi_cleaned`.")
# print("✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_joined'.")


✅ Dataset berhasil digabungkan berdasarkan `query` dengan struktur mengikuti `mr_tydi_cleaned`.


In [9]:
from datasets import DatasetDict

# Fungsi untuk menghapus rows dengan answers = None
def remove_none_answers(dataset):
    return dataset.filter(lambda row: row["answers"] is not None)

# Buat dataset baru tanpa row yang memiliki answers = None
finished_dataset = DatasetDict({
    "train": remove_none_answers(final_dataset["train"]),
    "dev": remove_none_answers(final_dataset["dev"]),
    "test": remove_none_answers(final_dataset["test"])
})

print("✅ Semua row dengan 'answers = None' telah dihapus dari dataset baru `finished_dataset`.")

Filter:   0%|          | 0/4902 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1224 [00:00<?, ? examples/s]

Filter:   0%|          | 0/829 [00:00<?, ? examples/s]

✅ Semua row dengan 'answers = None' telah dihapus dari dataset baru `finished_dataset`.


In [10]:
# Fungsi untuk mengecek apakah ada None dalam kolom 'answers'
def check_none_answers(dataset, split_name):
    none_count = sum(1 for row in dataset if row["answers"] is None)
    print(f"🔍 Split '{split_name}': {none_count} row(s) memiliki 'answers' = None")

# Periksa setiap split
check_none_answers(finished_dataset["train"], "train")
check_none_answers(finished_dataset["dev"], "dev")
check_none_answers(finished_dataset["test"], "test")

🔍 Split 'train': 0 row(s) memiliki 'answers' = None
🔍 Split 'dev': 0 row(s) memiliki 'answers' = None
🔍 Split 'test': 0 row(s) memiliki 'answers' = None


In [16]:
from transformers import AutoTokenizer, AutoModel

# # Load dataset yang sudah dibersihkan
# finished_dataset = load_from_disk("generated_data/mr_tydi_tydiqa_cleaned")

# Load tokenizer & model untuk Multilingual-E5-Small
model_name = "intfloat/multilingual-e5-small"
embedding_tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name).to("cuda:0")

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [17]:
import torch.nn.functional as F
import torch
from torch import Tensor
from tqdm import tqdm
import gc
from datasets import load_from_disk


# Fungsi average pooling
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Fungsi untuk memilih top 4 negative_passages berdasarkan similarity
def select_top4_negative_passages(example):
    query_text = example["query"]
    negative_passages = example["negative_passages"]

    # Jika sudah <= 4, tidak perlu pemrosesan
    if len(negative_passages) <= 4:
        return example

    # Ambil teks dari negative_passages
    neg_texts = [neg["text"] for neg in negative_passages]

    # Tokenisasi dan embedding query serta negative_passages
    batch_dict = embedding_tokenizer([query_text] + neg_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')
    batch_dict = {k: v.to("cuda:0") for k, v in batch_dict.items()}

    with torch.no_grad():
        outputs = embedding_model(**batch_dict)

    # Hitung embedding dan normalisasi
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)  # Normalisasi untuk cosine similarity

    # Hitung similarity scores (query vs negative_passages)
    query_embedding = embeddings[0].unsqueeze(0)  # Query ada di indeks pertama
    neg_embeddings = embeddings[1:]  # Negative passages setelah query
    scores = (query_embedding @ neg_embeddings.T).squeeze(0)  # Cosine similarity

    # Ambil indeks top 4 dengan similarity tertinggi
    top_indices = torch.argsort(scores, descending=True)[:4]

    # Simpan hanya 4 negative_passages terbaik
    example["negative_passages"] = [negative_passages[i] for i in top_indices]

    # Bersihkan cache GPU setelah query diproses
    del batch_dict, outputs, embeddings, scores
    torch.cuda.empty_cache()
    gc.collect()

    return example

# Terapkan fungsi ke split train
finished_dataset["train"] = finished_dataset["train"].map(select_top4_negative_passages)

# Simpan hasilnya ke disk
finished_dataset.save_to_disk("generated_data/mr_tydi_tydiqa_final")

print("✅ Negative passages pada split train telah dipangkas menjadi top 4 berdasarkan similarity!")
print("✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_final'.")


Map:   0%|          | 0/4542 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4542 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1143 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/565 [00:00<?, ? examples/s]

✅ Negative passages pada split train telah dipangkas menjadi top 4 berdasarkan similarity!
✅ Dataset telah disimpan di 'generated_data/mr_tydi_tydiqa_final'.


# Uji LLAMA 3.2 3B vs Groq

In [2]:
from datasets import load_from_disk
from utils import load_model_and_tokenizer

model_name = "meta-llama/Llama-3.2-3B-Instruct"
model, tokenizer, config = load_model_and_tokenizer(model_name)
finished_dataset = load_from_disk("./generated_data/mr_tydi_tydiqa_final")

Loading model on cuda with torch.float16


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from typing import List, Dict
from utils import DeviceType, format_chat_prompt, prepare_inputs
from transformers import AutoTokenizer
import torch

def combine_passages(example):
    """
    Menggabungkan positive_passages dan negative_passages menjadi satu list.
    """
    example["all_passages"] = example["positive_passages"] + example["negative_passages"]
    return example

def format_passages(example):
    """
    Memformat top 3 passages menjadi format "Judul: {title} \nTeks: {text}".
    """
    formatted_passages = [
        f"Judul: {p['title']} \nTeks: {p['text']}" for p in example["all_passages"][:3]  # Ambil 3 passage pertama
    ]
    return "\n\n".join(formatted_passages)

def truncate_for_flan_t5(messages: List[Dict[str, str]], tokenizer_llama, max_flan_tokens=512):
    """
    Truncate input agar memiliki konten utama yang setara dengan 512 token di Flan-T5, lalu tambahkan token spesial LLaMA.

    Args:
        messages: List chat messages (role: "user", "assistant", dll).
        tokenizer_llama: Tokenizer untuk model LLaMA.
        max_flan_tokens: Jumlah token utama yang harus sesuai dengan input Flan-T5.

    Returns:
        str: Prompt yang sudah di-truncate dan siap digunakan untuk LLaMA.
    """
    # Format chat prompt sesuai tokenizer LLaMA
    formatted_prompt = format_chat_prompt(messages, tokenizer_llama)

    # Tokenisasi prompt dengan tokenizer LLaMA
    tokenized = tokenizer_llama(formatted_prompt, return_tensors="pt", truncation=False)["input_ids"][0]

    # Hitung jumlah token spesial yang ditambahkan oleh tokenizer LLaMA
    special_tokens_count = len(tokenized) - sum(len(tokenizer_llama(m["content"])["input_ids"]) for m in messages)

    # Pastikan hanya konten utama yang sesuai dengan batas Flan-T5
    max_llama_tokens = max_flan_tokens  # Konten utama tetap 512 token

    # Truncate teks utama sebelum ditambahkan token spesial
    truncated_tokenized = tokenized[:max_llama_tokens]  # Potong agar isi utama setara Flan-T5

    # Decode kembali untuk memastikan input tetap masuk akal
    truncated_text = tokenizer_llama.decode(truncated_tokenized, skip_special_tokens=False)

    return truncated_text, special_tokens_count

# 🔍 Menerapkan pipeline preprocessing ke dataset (minimal 3 row untuk pengecekan konsistensi special token count)
processed_dataset = finished_dataset.map(combine_passages).map(lambda example: {
    "formatted_passages": format_passages(example),
    "question": example["query"]
})

# Ambil minimal 3 row untuk diuji
test_rows = processed_dataset['train'].select(range(min(3, len(processed_dataset))))

special_token_counts = []
for row in test_rows:
    messages = [
        {"role": "user", "content": f'{row["formatted_passages"]}\n\nRingkaslah teks di atas agar dapat menjawab pertanyaan secara mendetail. Jangan berikan pengantar pada hasil. Pertanyaan: "{row["question"]}"'}
    ]
    
    truncated_text, special_tokens_count = truncate_for_flan_t5(messages, tokenizer)
    special_token_counts.append(special_tokens_count)

    print(f"🔹 Truncated input for LLaMA:\n{truncated_text}\n")
    print(f"🔸 Special Token Count: {special_tokens_count}\n")
    
# 🔍 Cek apakah jumlah token spesial konsisten
print(f"📊 Konsistensi Special Token Count: {special_token_counts}")


🔹 Truncated input for LLaMA:
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 09 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Judul: Ernest Douwes Dekker 
Teks: Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.

Judul: Eduard Douwes Dekker 
Teks: Eduard Douwes Dekker kemudian pindah ke Ingelheim am Rhein dekat Sungai Rhein sampai akhirnya meninggal 19 Februari 1887.

Judul: Ernest Douwes Dekker 
Teks: Dr. Ernest François Eugène Douwes Dekker (umumnya dikenal dengan nama Douwes Dekker atau Danudirja Setiabudi; ) adalah seorang pejuang kemerdekaan dan pahlawan nasional Indonesia.

Ringkaslah teks di atas agar dapat menjawab pertanyaan secara mendetail. Jangan berikan pengantar pada hasil. Pertanyaan: "dimanakah Dr. Ernest François Eugène Douwes Dekker meninggal?"<|eot_id|><|

In [19]:
finished_dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 4542
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 1143
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 565
    })
})

In [29]:
def get_special_tokens(messages, tokenizer_llama):
    """
    Menampilkan token spesial yang ditambahkan oleh tokenizer LLaMA dalam format teks dan angka (token ID).
    
    Args:
        messages: List pesan dalam format chat.
        tokenizer_llama: Tokenizer dari model LLaMA.

    Returns:
        Dict berisi daftar token spesial dalam bentuk teks (string) dan ID token.
    """
    # Format prompt dengan tokenizer LLaMA
    formatted_prompt = format_chat_prompt(messages, tokenizer_llama)

    # Tokenisasi seluruh prompt
    full_tokenized = tokenizer_llama(formatted_prompt, return_tensors="pt", truncation=False)["input_ids"][0].tolist()

    # Tokenisasi ulang hanya bagian konten utama (tanpa format chat)
    content_only = "".join([m["content"] for m in messages])  # Gabungkan konten
    content_tokenized = tokenizer_llama(content_only, return_tensors="pt", truncation=False)["input_ids"][0].tolist()

    # Token spesial = Token yang ada di full_tokenized tetapi tidak ada di content_tokenized
    special_token_ids = [tid for tid in full_tokenized if tid not in content_tokenized]
    
    # Konversi token spesial dari ID ke teks dalam satu string
    special_tokens_text = " ".join(tokenizer_llama.convert_ids_to_tokens(special_token_ids))

    return {
        "special_token_ids": special_token_ids,
        "special_tokens_text": special_tokens_text,  # Sekarang dalam format string
        "num_special_tokens": len(special_token_ids)  # Tambahkan jumlah token spesial
    }

# 🔍 Cek 5 row pertama dari finished_dataset
special_tokens_per_row = []
num_rows = min(10, len(finished_dataset['train']))  # Ambil maksimal 5 row
processed_dataset = finished_dataset['train'].select(range(num_rows)).map(combine_passages).map(lambda example: {
    "formatted_passages": format_passages(example),
    "question": example["query"]
})

token_counts = []  # Untuk menyimpan jumlah token spesial setiap row

for i in range(num_rows):
    row = processed_dataset[i]
    messages = [
        {"role": "user", "content": f'{row["formatted_passages"]}\n\nRingkaslah teks di atas agar dapat menjawab pertanyaan secara mendetail. Jangan berikan pengantar pada hasil. Pertanyaan: "{row["question"]}"'}
    ]
    
    special_tokens = get_special_tokens(messages, tokenizer)
    special_tokens_per_row.append(special_tokens)
    token_counts.append(special_tokens["num_special_tokens"])

    print(f"🔹 Row {i+1} - Token Spesial dalam format ID: {special_tokens['special_token_ids']}")
    print(f"🔸 Row {i+1} - Token Spesial dalam format teks: \"{special_tokens['special_tokens_text']}\"")  # Ditampilkan dalam satu string
    print(f"🔢 Row {i+1} - Jumlah Token Spesial: {special_tokens['num_special_tokens']}\n")

# 🔍 Periksa apakah semua row memiliki token spesial yang sama
all_same = all(special_tokens_per_row[i]["special_token_ids"] == special_tokens_per_row[0]["special_token_ids"]
               for i in range(1, num_rows))

# 🔢 Tampilkan jumlah token spesial di semua row
print("📊 Jumlah Token Spesial per Row:", token_counts)

print("\n✅ Semua row memiliki token spesial yang sama" if all_same else "❌ Token spesial berbeda di beberapa row.")


🔹 Row 1 - Token Spesial dalam format ID: [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 6790, 2366, 18, 198, 15724, 2696, 2545, 13806, 2366, 20, 271, 128009, 128006, 882, 128007, 271, 128009, 128006, 78191, 128007, 271]
🔸 Row 1 - Token Spesial dalam format teks: "<|start_header_id|> system <|end_header_id|> ĊĊ Cut ting ĠKnowledge ĠDate ĠDecember 202 3 Ċ Today ĠDate 09 ĠFeb 202 5 ĊĊ <|eot_id|> <|start_header_id|> user <|end_header_id|> ĊĊ <|eot_id|> <|start_header_id|> assistant <|end_header_id|> ĊĊ"
🔢 Row 1 - Jumlah Token Spesial: 29

🔹 Row 2 - Token Spesial dalam format ID: [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 6790, 2366, 198, 15724, 2696, 2545, 13806, 2366, 20, 271, 128009, 128006, 882, 128007, 271, 128009, 128006, 78191, 128007, 271]
🔸 Row 2 - Token Spesial dalam format teks: "<|start_header_id|> system <|end_header_id|> ĊĊ Cut ting ĠKnowledge ĠDate ĠDecember 202 Ċ Today ĠDate 09 ĠFeb 202 5 ĊĊ <|eot_id|> <|start_header_id|> user <|end_header_id|> ĊĊ <|eot_id|

In [24]:
num_rows = min(5, len(finished_dataset))
num_rows

3

In [5]:
import random

SEED = 42
random.seed(SEED)

# Ambil sampel dari tiap split
train_sample = random.sample(list(finished_dataset["train"]), 132)
dev_sample = random.sample(list(finished_dataset["dev"]), 40)
test_sample = random.sample(list(finished_dataset["test"]), 28)

# Gabungkan semua sampel
sample_data = train_sample + dev_sample + test_sample

print(f"Total samples: {len(sample_data)}")  # Harusnya 71 (50+13+8)

def combine_passages(example):
    example["all_passages"] = example["positive_passages"] + example["negative_passages"]
    return example

# Terapkan ke semua sampel
sample_data = [combine_passages(ex) for ex in sample_data]

def format_passages(example):
    formatted_passages = [
        f"Judul: {p['title']} \nTeks: {p['text']}" for p in example["all_passages"][:3]  # Ambil 5 passage
    ]
    return "\n\n".join(formatted_passages)  # Gabungkan semua passages

# Tambahkan kolom "formatted_passages"
for ex in sample_data:
    ex["formatted_passages"] = format_passages(ex)

Total samples: 200


In [3]:
sample_data[0]['query']

'Dimana Kerajaan Kuru didirikan?'

In [6]:
from datasets import Dataset

# Konversi list sample_data menjadi Dataset
sample_dataset = Dataset.from_list(sample_data)
sample_dataset

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'all_passages', 'formatted_passages'],
    num_rows: 200
})

In [ ]:
from summarize import summarize_top_5_combined
import time
import json

model_name = "meta-llama/Llama-3.2-1B-Instruct"

start = time.time()
sample_output = summarize_top_5_combined(
    model_name=model_name,
    dataset=sample_dataset,
    query_col="query",
    docs_col="formatted_passages", 
    model=model, 
    tokenizer=tokenizer,
    batch_size=2
)
end = time.time()
duration = end - start
print(f"Durasi: {duration} detik")

# output_file = "generated_data/sample_summaries.json"

# # Konversi dataset ke list dan simpan sebagai JSON
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(sample_data.to_list(), f, ensure_ascii=False, indent=2)

# print(f"✅ Data telah disimpan dalam {output_file}")

# output_file_parquet = "generated_data/sample_summaries.parquet"

# # Simpan sebagai Parquet
# sample_data.to_parquet(output_file_parquet)

# print(f"✅ Data telah disimpan dalam {output_file_parquet}")


Summarizing dataset:   0%|          | 0/1 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Summarizing dataset: 100%|██████████| 1/1 [02:41<00:00, 161.79s/it]

Durasi: 162.0047287940979 detik


In [8]:
import time
import numpy as np
from utils import load_model_and_tokenizer
from summarize import summarize_top_5_combined
model_name = "meta-llama/Llama-3.2-3B-Instruct"

batch_sizes = [1, 2, 4, 8]  # Uji berbagai batch size
num_samples = 200  # Coba dengan 100 sample

results = {}
ds = []

for batch_size in batch_sizes:
    start = time.time()
    
    ds.append(summarize_top_5_combined(
        model_name="meta-llama/Llama-3.2-1B-Instruct",
        dataset=sample_dataset.select(range(num_samples)),  # Ambil subset data
        query_col="query",
        docs_col="formatted_passages",
        model=model, 
        tokenizer=tokenizer,
        batch_size=batch_size
    ))

    end = time.time()
    duration = end - start
    avg_time_per_instance = duration / num_samples
    
    results[batch_size] = avg_time_per_instance
    print(f"Batch size {batch_size}: {duration:.2f} detik ({avg_time_per_instance:.4f} detik per instance)")

# Menampilkan hasil
optimal_batch = min(results, key=results.get)
print(f"\n🚀 Batch size optimal: {optimal_batch} (dengan rata-rata waktu per instance: {results[optimal_batch]:.4f} detik)")

Summarizing dataset:   0%|          | 0/200 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.
Summarizing dataset: 100%|██████████| 200/200 [19:33<00:00,  5.87s/it]


Batch size 1: 1174.00 detik (5.8700 detik per instance)


Summarizing dataset: 100%|██████████| 100/100 [23:42<00:00, 14.23s/it]


Batch size 2: 1422.90 detik (7.1145 detik per instance)


Summarizing dataset: 100%|██████████| 50/50 [26:32<00:00, 31.84s/it]


Batch size 4: 1592.24 detik (7.9612 detik per instance)


Summarizing dataset: 100%|██████████| 25/25 [28:15<00:00, 67.83s/it]

Batch size 8: 1695.84 detik (8.4792 detik per instance)

🚀 Batch size optimal: 1 (dengan rata-rata waktu per instance: 5.8700 detik)


In [19]:
ds[0]

Dataset({
    features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers', 'all_passages', 'formatted_passages', 'summary'],
    num_rows: 200
})

In [23]:
for i in range(4):
    print('batch size', batch_sizes[i])
    for j in range(5):
        print('QUERY:', ds[i][j]['query'])
        print('TOP 3:-----------------------------------------')  
        print(ds[i][j]['formatted_passages'])
        print('-----------------------------------------------\nsummary:', ds[i][j]['summary'])
        print('answer:', ds[i][j]['answers'])
        print()
    print("============================================================================================================")

batch size 1
QUERY: Dimana Kerajaan Kuru didirikan?
TOP 3:-----------------------------------------
Judul: Kerajaan Kuru 
Teks: Dalam sastra dan wiracarita India Kuno, Kerajaan Kuru merupakan kerajaan yang diperintah oleh Wangsa Kuru, keturunan Sang Kuru. Tidak diketahui dengan pasti kapan kerajaan ini berdiri, dan hingga sekarang dikenal sebagai legenda dalam wiracarita India, seperti misalnya Mahabharata. Kerajaan Kuru yang lain berada di utara Himalaya, dan disebut Uttara Kuru. Menurut sastra Hindu, Kerajaan Kuru terbentang di antara sungai Saraswati dan sungai Gangga. Salah satu kitab yang dijadikan sumber keberadaan kerajaan Kuru adalah Mahabharata, dan tokoh utama yang diceritakan dalam kitab tersebut merupakan keturunan Kuru. Menurut Mahabharata, pada masa pemerintahan Raja Dretarastra, Kerajaan tersebut terbagi menjadi dua bagian, yaitu Kurujangala dan Kuru asli.

Judul: Kuru (raja) 
Teks: Semenjak kecil, Kuru dididik oleh seorang Maharesi Wasista. Pada usia sepuluh tahun, ia s

In [10]:
ds[1][0]['summary']

'Kerajaan Kuru didirikan di wilayah Himalaya, kemungkinan terletak di Tibet, utara Xin Jiang, atau Kirgizstan.'

In [ ]:
ds[2][0]

In [10]:
ds[0]['summary']

['Kerajaan Kuru didirikan di antara sungai Saraswati dan sungai Gangga.',
 'Ya, ada 5 tingkatan ekologi yang disebutkan dalam teks tersebut:\n\n1. Populasi\n2. Komunitas\n3. Ekosistem\n4. Ekologi Manusia\n5. Ekologi Global',
 'Pangeran Natakusuma I (Panembahan Somala) adalah pendiri Keraton Sumenep.',
 'Masashi Kishimoto pertama kali berkarya sebagai mangaka semenjak usia SD.',
 'Sebuah sistem adalah suatu kesatuan yang terdiri komponen atau elemen yang dihubungkan bersama untuk memudahkan aliran informasi, materi atau energi untuk mencapai suatu tujuan.',
 'Unsur "A" pada Jalur ABG merupakan Angkatan Bersenjata Republik Indonesia (ABRI).',
 'Tidak ada teks yang membahas tentang suku yang mendiami wilayah Surakarta. Teks yang ada membahas tentang wilayah Jawa Tengah dan beberapa suku yang mendiami wilayah Rejang Lebong, Sum',
 'Permainan daring multipemain masif (MMO) adalah permainan daring dengan jumlah pemain yang besar, biasanya dari ratusan sampai ribuan, di server yang sama.',
 '

In [13]:
ds[0]['answers']

['sungai Saraswati dan sungai Gangga',
 'populasi, komunitas, dan ekosistem yang saling memengaruhi dan merupakan suatu sistem yang menunjukkan kesatuan',
 'Panembahan Somala',
 'usia SD',
 'suatu kesatuan yang terdiri komponen atau elemen yang dihubungkan bersama untuk memudahkan aliran informasi, materi atau energi untuk mencapai suatu tujuan',
 'Angkatan Bersenjata Republik Indonesia',
 'Suku Jawa',
 'dapat diakses oleh banyak pemain secara massal untuk bermain bersama dalam dunia maya yang terus berkembang pada saat yang sama melalui sambungan Internet dan LAN',
 'struktur di dalam sel berupa deret panjang molekul yang terdiri dari satu molekul DNA[1] dan berbagai protein terkait yang merupakan informasi genetik suatu organisme',
 '78,772km']

In [8]:
import torch

# GPU yang digunakan
def check_gpu():
    device = torch.device("cuda:0")

    # Total memori GPU
    total_memory = torch.cuda.get_device_properties(device).total_memory / (1024 ** 3)  # Dalam GB
    print(f"Total GPU Memory: {total_memory:.2f} GB")

    # Memori yang sudah dialokasikan oleh PyTorch
    allocated_memory = torch.cuda.memory_allocated(device) / (1024 ** 3)  # Dalam GB
    print(f"Allocated GPU Memory: {allocated_memory:.2f} GB")

    max_reserved_memory = torch.cuda.max_memory_reserved(device) / (1024 ** 3)  # Dalam GB
    print(f"Max Reserved GPU Memory: {max_reserved_memory:.2f} GB")

    # Memori GPU yang dicadangkan oleh PyTorch
    reserved_memory = torch.cuda.memory_reserved(device) / (1024 ** 3)  # Dalam GB
    print(f"Reserved GPU Memory: {reserved_memory:.2f} GB")

    # Memori GPU yang tersedia
    free_memory = reserved_memory - allocated_memory
    print(f"Free GPU Memory: {free_memory:.2f} GB")

In [9]:
check_gpu()

Total GPU Memory: 6.00 GB
Allocated GPU Memory: 5.99 GB
Max Reserved GPU Memory: 9.83 GB
Reserved GPU Memory: 6.05 GB
Free GPU Memory: 0.06 GB


In [10]:
torch.cuda.empty_cache()

In [12]:
finished_dataset

DatasetDict({
    train: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 4542
    })
    dev: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 1143
    })
    test: Dataset({
        features: ['query_id', 'query', 'positive_passages', 'negative_passages', 'tydiqa_id', 'answers'],
        num_rows: 565
    })
})

In [25]:
sample_output['summary']

['Kerajaan Kuru didirikan di wilayah Himalaya timur, kemungkinan terletak di Uttarakhand atau di Tibet.']

In [26]:
sample_output['answers']

['sungai Saraswati dan sungai Gangga']

In [30]:
print(sample_output['formatted_passages'])

['Judul: Kerajaan Kuru \nTeks: Dalam sastra dan wiracarita India Kuno, Kerajaan Kuru merupakan kerajaan yang diperintah oleh Wangsa Kuru, keturunan Sang Kuru. Tidak diketahui dengan pasti kapan kerajaan ini berdiri, dan hingga sekarang dikenal sebagai legenda dalam wiracarita India, seperti misalnya Mahabharata. Kerajaan Kuru yang lain berada di utara Himalaya, dan disebut Uttara Kuru. Menurut sastra Hindu, Kerajaan Kuru terbentang di antara sungai Saraswati dan sungai Gangga. Salah satu kitab yang dijadikan sumber keberadaan kerajaan Kuru adalah Mahabharata, dan tokoh utama yang diceritakan dalam kitab tersebut merupakan keturunan Kuru. Menurut Mahabharata, pada masa pemerintahan Raja Dretarastra, Kerajaan tersebut terbagi menjadi dua bagian, yaitu Kurujangala dan Kuru asli.\n\nJudul: Kuru (raja) \nTeks: Semenjak kecil, Kuru dididik oleh seorang Maharesi Wasista. Pada usia sepuluh tahun, ia sudah menguasai sastra, kitab suci, dan tugas-tugas sebagai seorang raja. Menurut kitab "Wamana